<a href="https://colab.research.google.com/github/paulodowd/EMATM0054_53/blob/main/SL5_ResultsOverSerial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supplementary Labsheet 5: Results over Serial

This labsheet will help you to record results (measurements) on your 3Pi+ robot, and then retrieve them later over the Serial connection.  This will be a vital part of your work for **Assessment 2**.  The summary of the approach is to:
- create an array as a **global variable**.
- add results to this array over time, at a fixed time interval.
- stop the robot, and report the content of the array via `Serial.print()`.



<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/tick.png" align="left"> A core assessment criteria of Assessment 2 is to demonstrate your ability to analyse and evaluate a robotic system.  The primary means to do demonstrate analysis and evaluation will be through capturing measurements of your system, producing plots (graphs), and discussing them in written communication.  




<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/info.png" align="left"> The 3Pi+ has a known issue: Unplugging the USB cable during the robot operation can cause irratic behaviour, and/or a reset of your robot.  However, you can activate your robot **without the USB cable plugged in**, and then **plug it in later** without resetting the robot. Your robot may still reset if you close the Serial monitor.



<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/info.png" align="left"> The 3Pi+ robot also has non-volatile EEPROM memory.  Support for this has been removed because it has very limited utility.  EEPROM provides 1000 bytes of storage.  However, a `float` variable alone takes 4 bytes each.  Therefore, the recommended approach is the one detailed in this labsheet.

The below flow chart illustrates the overall process of capturing results:

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/ResultsOverSerial_FlowChart.png">
</p>

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/info.png" align="left"> You will want to structure your program for the 3Pi+ robot to have at least two states.  **First**, the code that operates your robot behaviours as your experiment trial.  **Second**, a state that stops the robot behaviours, and continuously reports the results over `Serial.print()`.  To run another trial of your robot, you would need to reset the power to the robot.

<hr><br><br><br><br>

## Example Code

Review the below example code.  This code is incomplete where you will need to add the code that operates your 3Pi+ robot behaviours.  You will also need to consider which measurements you are taking, and so adapt how global variables are used for this purpose.





<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/info.png" align="left"> This example records 100 values of $x$, $y$ and $\theta$, and so creates an array of the form `results[100][3]`.   This could also be achieved with three 1 dimensional arrays, such as `float x_results[100]; float y_results[100]; float theta_results[100];`.

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/info.png" align="left"> The total number of results you will be able to record will be constrained by the overall memory use of your robot controller code.  When you compile your code, check the output of the compiler for an estimate of the memory use.

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/stop.png" align="left"> The compiler will tell you an estimate of memory usage for global variables.  For example, `Global variables use 710 bytes (27%) of dynamic memory, leaving 1850 bytes for local variables. Maximum is 2560 bytes.`  We can see that the maximum is **2560 bytes**.  The compiler may estimate incorrectly.  If you use all dynamic memory, **your robot will likely reset or behave irratically**.  Some processes (such as `Serial.print()`) are also using dynamic memory, which is very difficult for the compiler to predict.  Therefore, try to stay within 2560 bytes with space remaining.

<img src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53/main/Images/info.png" align="left"> If you need to record more results than you have memory, you may need to run repeated trials that focus on only 1 result (or subset) at a time.  For example, 10 repeated trials that capture $x$, then 10 repeated trials that capture $y$, and so on.   The design of your experiment is important - we want to control the factors that may influence results as much as possible.


```c
// We will store our results as a global
// variable. Just as an example, let's store
// 100 x,y, theta values.
// Note, this means 100 * 3 floats. A float is
// represented by 4 bytes.  Therefore,
// 100 * 3 * 4 bytes (1200bytes).
#define MAX_RESULTS 100
#define VARIABLES   3
float results[ MAX_RESULTS ][ VARIABLES ];

// We will use this to track our position in
// the results[][] array over time.  It will
// start at 0, and increase to MAX_RESULTS.
int results_index;

// State machine.  We need at least two.
#define STATE_RUNNING_EXPERIMENT  0
#define STATE_FINISHED_EXPERIMENT 1
int state;

// Time stamp to track whether the experiment
// duration has elapsed.
unsigned long experiment_start_ts;
#define EXPERIMENT_END_MS 5000 // 5 seconds?

// We can try to automate the time interval
// of storing results.
unsigned long record_results_ts;
unsigned long results_interval_ms;


void setup() {

  // Make sure we start storing results at
  // position 0 in the results[][] array.
  results_index = 0;

  // Work out the time interval for storing
  // results, given we have a max experiment
  // duration, and a maximum number of results
  results_interval_ms = ( EXPERIMENT_END_MS / MAX_RESULTS );
  
  // Start robot in running experiment state.
  state = STATE_RUNNING_EXPERIMENT;

  // Set to start recording results into
  // row 0 of our results array
  results_index = 0;

  // Activate the serial port for later.
  Serial.begin(9600);
  delay(1000);
  
  // Beep so we know when robot has activated
  // or reset.
  analogWrite( BUZZ_PIN, 120 );
  delay( 200 );
  analogWrite( BUZZ_PIN, 0 );

  // Record start time into timestamps
  experiment_start_ts millis();
  record_results_ts = millis();
  
}


void loop() {

  if( state == STATE_RUNNING_EXPERIMENT ) {

    // Your experiment program would need
    // to go here. E.g., whatever behaviours
    // you are investigating as a trial of
    // your experiment.


    // Is it time to record a result?
    unsigned long elapsed_time;
    elapsed_time = millis() - record_results_ts;
    
    if( elapsed_time > results_interval_ms ) {

        // Move time stamp forwards for next
        // iteration.
        record_results_ts = millis();

        // Let's be safe and check we haven't
        // filled up the results array already.
        if( results_index < MAX_RESULTS ) {
          
          results[ results_index ][0] = 0; // save x (from kinematics?)
          results[ results_index ][1] = 0; // save y (from kinematics?)
          results[ results_index ][2] = 0; // save theta (from kinematics?)

          // Increment result index for next time.
          results_index++;

        } else {
          // If RESULTS_MAX has been reached, if yes
          // then we have also finished the experiment.
          state = STATE_FINISHED_EXPERIMENT;
          return;
        }

    }


    // Has the experiment duration been reached?
    // If yes, move into the second state to
    // report the results.
    elapsed_time = millis() - experiment_start_ts;
    if( elapsed_time > EXPERIMENT_END_MS ) {
      
      // Transition to other state.
      state = STATE_FINISHED_EXPERIMENT;
      return;
    }
    



  } else if ( state == STATE_FINISHED_EXPERIMENT) {

    // Once this state has been reached, the robot
    // will remain in this state.  This state just
    // prints out all the results, then delays 3
    // seconds so you can copy-paste from serial
    // monitor.

    // Stop robot from moving.
    motors.setPWM( 0, 0 ); // from Labsheet 1.

    // Loop through the results, printing them
    // line by line.  This formatting is CSV
    // (comma seperated values) and can be
    // imported to Excel, Python, etc.
    int result;
    Serial.println("Sample, X, Y, Theta\n");
    for( result = 0; result < MAX_RESULTS; result++ ) {
      Serial.print( result );
      Serial.print(",");
      Serial.print( results[ result ][0] ); // X
      Serial.print(",");
      Serial.print( results[ result ][1] ); // Y
      Serial.print(",");
      Serial.print( results[ result ][2] ); // Theta
      Serial.print("\n");
    }


    // A delay is useful so that you have time
    // to copy and paste from Serial Monitor.
    delay(3000);
      

  }

}



```

